In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/weather-data-for-covid19-data-analysis/training_data_with_weather_info_week_4.csv
/kaggle/input/weather-data-for-covid19-data-analysis/training_data_with_weather_info_week_1.csv
/kaggle/input/covid19-global-forecasting-week-4/train.csv
/kaggle/input/covid19-global-forecasting-week-4/submission.csv
/kaggle/input/covid19-global-forecasting-week-4/test.csv
/kaggle/input/newest-bed-api-for-each-country/Newest_avg_bed_API.csv
/kaggle/input/population/population_by_country_2020.csv
/kaggle/input/covid19/submit-XGB.csv


In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from google.cloud import bigquery
import matplotlib.pyplot as plt
import os

In [3]:
train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')
test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')

# Preprocessing

### fill the missing 'Province' with 'Country'

In [4]:
train.rename(columns = {'Province_State':'Province','Country_Region':'Country'},inplace = True)
test.rename(columns = {'Province_State':'Province','Country_Region':'Country'},inplace = True)

In [5]:
for i in range(len(train)):
    if train.Province[i] is np.NaN:
        train.Province[i] = train.Country[i]
for i in range(len(test)):
    if test.Province[i] is np.NaN:
        test.Province[i] = test.Country[i]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### transform date to days

In [6]:
def get_days(x):
    x = datetime.strptime(x,'%Y-%m-%d')
    first = datetime.strptime('2020-01-01','%Y-%m-%d')
    diff = (x - first).days
    return diff

In [7]:
train['Days_After_1stJan'] = train.Date.apply(lambda x :get_days(x))
test['Days_After_1stJan'] = test.Date.apply(lambda x :get_days(x))

In [8]:
train.head()

,Id,Province,Country,Date,ConfirmedCases,Fatalities,Days_After_1stJan
0,1,Afghanistan,Afghanistan,2020-01-22,0.0,0.0,21
1,2,Afghanistan,Afghanistan,2020-01-23,0.0,0.0,22
2,3,Afghanistan,Afghanistan,2020-01-24,0.0,0.0,23
3,4,Afghanistan,Afghanistan,2020-01-25,0.0,0.0,24
4,5,Afghanistan,Afghanistan,2020-01-26,0.0,0.0,25


In [9]:
test.head()

,ForecastId,Province,Country,Date,Days_After_1stJan
0,1,Afghanistan,Afghanistan,2020-04-02,92
1,2,Afghanistan,Afghanistan,2020-04-03,93
2,3,Afghanistan,Afghanistan,2020-04-04,94
3,4,Afghanistan,Afghanistan,2020-04-05,95
4,5,Afghanistan,Afghanistan,2020-04-06,96


### Add Date Features

In [10]:
def get_dt(x):
    x = datetime.strptime(x,'%Y-%m-%d')
    return x
def get_dayofweek(x):
    return x.dayofweek
def get_month(x):
    return x.month
def get_day(x):
    return x.day

In [11]:
train['Date'] = train.Date.apply(lambda x:get_dt(x))
test['Date'] = test.Date.apply(lambda x:get_dt(x))
train['Dayofweek'] = train.Date.apply(lambda x:get_dayofweek(x))
test['Dayofweek'] = test.Date.apply(lambda x:get_dayofweek(x))
train['Month'] = train.Date.apply(lambda x:get_month(x))
test['Month'] = test.Date.apply(lambda x:get_month(x))
train['Day'] = train.Date.apply(lambda x:get_day(x))
test['Day'] = test.Date.apply(lambda x:get_day(x))

# Merge population information for each country

In [12]:
country_info = pd.read_csv('/kaggle/input/population/population_by_country_2020.csv')

### Select wanted features from table

In [13]:
population = pd.DataFrame(country_info.iloc[:,[0,1,4,5,6,8,9]])
population.columns = ['Country','Population','Density','Land_Area','Migrants','MedAge','UrbanPopRate']

### fill NAN with median and mode

In [14]:
for i in range(len(population)):
    if np.isnan(population.Migrants[i]):
        population.Migrants[i] = np.nanmedian(population.Migrants)
    if population.MedAge[i] == 'N.A.':
        population.MedAge[i] = 19
    if population.UrbanPopRate[i] == 'N.A.':
        population.UrbanPopRate[i] = '57%'

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### Collect data for countries without population info manually

In [15]:
kosovo = pd.DataFrame([['Kosovo'],[2000700],[168],[10887],[0],[19],['57%']])
kosovo = kosovo.T
kosovo.columns = population.columns


population = population.append(kosovo)

In [16]:
westbank = pd.DataFrame([['West Bank and Gaza'],[2697687],[485],[5559],[0],[19],['57%']])
westbank = westbank.T
westbank.columns = population.columns

population = population.append(westbank)

In [17]:
DP = pd.DataFrame(['Diamond Princess',2666,191522,0.01392,2666,19,'100%'])
DP = DP.T
DP.columns = population.columns

population = population.append(DP)

In [18]:
Shangdan = pd.DataFrame(['MS Zaandam',1432,189618,0.007552,1432,19,'100%'])
Shangdan = Shangdan.T
Shangdan.columns = population.columns

population = population.append(Shangdan)

### Change names of countries to match names in train dataframe

In [19]:
Congo = population[population.Country == 'Congo']
Congo['Country'] = 'Congo (Brazzaville)'
new1 = Congo.copy()
Congo['Country'] = 'Congo (Kinshasa)'
new2 = Congo.copy()

population = population.append(new1)
population = population.append(new2)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [20]:
population.Country[population.Country == 'United States'] = 'US'
population.Country[population.Country == 'Taiwan'] = 'Taiwan*'
population.Country[population.Country == 'South Korea'] = 'Korea, South'
population.Country[population.Country == 'Côte d\'Ivoire'] = 'Cote d\'Ivoire'
population.Country[population.Country == 'Czech Republic (Czechia)'] = 'Czechia'
population.Country[population.Country == 'Myanmar'] = 'Burma'
population.Country[population.Country == 'St. Vincent & Grenadines'] = 'Saint Vincent and the Grenadines'
population.Country[population.Country == 'Saint Kitts & Nevis']  = 'Saint Kitts and Nevis'
population.Country[population.Country == 'Sao Tome & Principe']  = 'Sao Tome and Principe'

### merge df

In [21]:
train = pd.merge(train,population,left_on = 'Country',right_on='Country',how='left')
test = pd.merge(test,population,left_on = 'Country',right_on='Country',how = 'left')

In [22]:
train.head()

,Id,Province,Country,Date,ConfirmedCases,Fatalities,Days_After_1stJan,Dayofweek,Month,Day,Population,Density,Land_Area,Migrants,MedAge,UrbanPopRate
0,1,Afghanistan,Afghanistan,2020-01-22,0.0,0.0,21,2,1,22,38928346,60,652860,-62920,18,25%
1,2,Afghanistan,Afghanistan,2020-01-23,0.0,0.0,22,3,1,23,38928346,60,652860,-62920,18,25%
2,3,Afghanistan,Afghanistan,2020-01-24,0.0,0.0,23,4,1,24,38928346,60,652860,-62920,18,25%
3,4,Afghanistan,Afghanistan,2020-01-25,0.0,0.0,24,5,1,25,38928346,60,652860,-62920,18,25%
4,5,Afghanistan,Afghanistan,2020-01-26,0.0,0.0,25,6,1,26,38928346,60,652860,-62920,18,25%


In [23]:
test.head()

,ForecastId,Province,Country,Date,Days_After_1stJan,Dayofweek,Month,Day,Population,Density,Land_Area,Migrants,MedAge,UrbanPopRate
0,1,Afghanistan,Afghanistan,2020-04-02,92,3,4,2,38928346,60,652860,-62920,18,25%
1,2,Afghanistan,Afghanistan,2020-04-03,93,4,4,3,38928346,60,652860,-62920,18,25%
2,3,Afghanistan,Afghanistan,2020-04-04,94,5,4,4,38928346,60,652860,-62920,18,25%
3,4,Afghanistan,Afghanistan,2020-04-05,95,6,4,5,38928346,60,652860,-62920,18,25%
4,5,Afghanistan,Afghanistan,2020-04-06,96,0,4,6,38928346,60,652860,-62920,18,25%


# Add weather information

In [24]:
temperature = pd.read_csv('/kaggle/input/weather-data-for-covid19-data-analysis/training_data_with_weather_info_week_4.csv')
temperature.rename(columns = {'Province_State':'Province','Country_Region':'Country'},inplace = True)

In [25]:
# fill NAN of Province with Country name
for i in range(len(temperature)):
    if temperature.Province[i] is np.NaN:
        temperature.Province[i] = temperature.Country[i]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
#delete useless features 
del temperature['ConfirmedCases']
del temperature['Fatalities']
del temperature['country+province']
del temperature['day_from_jan_first']

In [27]:
#transform date to datetime type
temperature.Date = temperature.Date.apply(lambda x:get_dt(x))

### since the weather data only contains information up to 2020-04-08,I delete the data of last 4days in the train dataframe

In [28]:
day1 = datetime.strptime('2020-04-09','%Y-%m-%d')
day2 = datetime.strptime('2020-04-10','%Y-%m-%d')
day3 = datetime.strptime('2020-04-11','%Y-%m-%d')
day4 = datetime.strptime('2020-04-12','%Y-%m-%d')

In [29]:
index_delete = []
for i in range(len(train)):
    if (train.Date[i] == day1) or (train.Date[i] == day2) or (train.Date[i] == day3) or (train.Date[i] == day4):
        index_delete.append(i)
        
train = (train.drop(index = index_delete)).reset_index(drop = True)

In [30]:
train = pd.merge(train,temperature,on=['Country','Province','Date'],how='left')

In [31]:
#fill NAN
train['Lat'][train['Lat'].isnull()] = np.nanmedian(train['Lat'])
train['Long'][train['Long'].isnull()] = np.nanmedian(train['Long'])
train['temp'][train['temp'].isnull()] = np.nanmedian(train['temp'])
train['min'][train['min'].isnull()] = np.nanmedian(train['min'])
train['max'][train['max'].isnull()] = np.nanmedian(train['max'])
train['slp'][train['slp'].isnull()] = np.nanmedian(train['slp'])
train['dewp'][train['dewp'].isnull()] = np.nanmedian(train['dewp'])
train['rh'][train['rh'].isnull()] = np.nanmedian(train['rh'])
train['ah'][train['ah'].isnull()] = np.nanmedian(train['ah'])
train['stp'][train['stp'].isnull()] = np.nanmedian(train['stp'])
train['wdsp'][train['wdsp'].isnull()] = np.nanmedian(train['wdsp'])
train['prcp'][train['prcp'].isnull()] = np.nanmedian(train['prcp'])
train['fog'][train['fog'].isnull()] = np.nanmedian(train['fog'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys

### Use auto arima to forecast weather data

In [32]:
!pip install pmdarima

     |████████████████████████████████| 1.5 MB 7.9 MB/s 


In [33]:
import pmdarima

In [34]:
#the outlier 'inf' will make the auto_arima come to an error,so it's replaced by 0
train.replace(np.inf,0,inplace=True)

In [35]:
#Using arima to predict the weather information for future
date_pred_df = pd.DataFrame(sorted(list(set(test.Date))),columns=['Date'])
date_pred_df = date_pred_df[7:]
date_pred_df.reset_index(inplace = True)
del date_pred_df['index']

nperiods = (datetime.strptime('2020-05-14','%Y-%m-%d')-datetime.strptime('2020-04-08','%Y-%m-%d')).days

weather_feature = ['temp', 'min', 'max', 'stp', 'slp', 'dewp', 'rh', 'ah','wdsp', 'prcp', 'fog']
weather_pred = pd.DataFrame(columns=['Date','temp', 'min', 'max', 'stp', 'slp', 'dewp', 'rh', 'ah','wdsp', 'prcp', 'fog'])
for prov in list(set(train.Province)):
    df = train[train.Province == prov]
    province_pred = date_pred_df.copy()
    for feature in weather_feature:
        ts = df[feature]
        model = pmdarima.auto_arima(ts)
        pred = model.predict(n_periods = nperiods)
        province_pred[feature] = pred
    province_pred['Province'] = prov
    weather_pred = pd.concat([weather_pred,province_pred],axis = 0)

/opt/conda/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/opt/conda/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/opt/conda/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimiza

In [36]:
test

,ForecastId,Province,Country,Date,Days_After_1stJan,Dayofweek,Month,Day,Population,Density,Land_Area,Migrants,MedAge,UrbanPopRate
0,1,Afghanistan,Afghanistan,2020-04-02,92,3,4,2,38928346,60,652860,-62920,18,25%
1,2,Afghanistan,Afghanistan,2020-04-03,93,4,4,3,38928346,60,652860,-62920,18,25%
2,3,Afghanistan,Afghanistan,2020-04-04,94,5,4,4,38928346,60,652860,-62920,18,25%
3,4,Afghanistan,Afghanistan,2020-04-05,95,6,4,5,38928346,60,652860,-62920,18,25%
4,5,Afghanistan,Afghanistan,2020-04-06,96,0,4,6,38928346,60,652860,-62920,18,25%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13454,13455,Zimbabwe,Zimbabwe,2020-05-10,130,6,5,10,14862924,38,386850,-116858,19,38%
13455,13456,Zimbabwe,Zimbabwe,2020-05-11,131,0,5,11,14862924,38,386850,-116858,19,38%
13456,13457,Zimbabwe,Zimbabwe,2020-05-12,132,1,5,12,14862924,38,386850,-116858,19,38%
13457,13458,Zimbabwe,Zimbabwe,2020-05-13,133,2,5,13,14862924,38,386850,-116858,19,38%


In [37]:
for i in range(len(test)):
    if test.Date[i]<datetime.strptime('2020-04-09','%Y-%m-%d'):
        test.drop(i,inplace=True)

In [38]:
#get longitude and latitude dataframe and merge it into test df
df_longlat = pd.DataFrame(columns = ['Province','Lat','Long'])
for i in range(len(train)):
    if train.Province[i] not in list(df_longlat.Province):
        df_longlat = df_longlat.append(train.iloc[i][['Province','Lat','Long']])
        
test = pd.merge(test,df_longlat,on = 'Province',how = 'left')

In [39]:
#adding weather feature to test data
test = pd.merge(test,weather_pred,on = ['Province','Date'],how = 'left')

In [40]:
train.head()

,Id_x,Province,Country,Date,ConfirmedCases,Fatalities,Days_After_1stJan,Dayofweek,Month,Day,...,min,max,stp,slp,dewp,rh,ah,wdsp,prcp,fog
0,1,Afghanistan,Afghanistan,2020-01-22,0.0,0.0,21,2,1,22,...,33.6,54.9,999.9,1024.3,27.4,0.545709,0.186448,9.4,0.00,0.0
1,2,Afghanistan,Afghanistan,2020-01-23,0.0,0.0,22,3,1,23,...,32.7,55.9,999.9,1020.8,22.8,0.461259,0.163225,14.9,99.99,1.0
2,3,Afghanistan,Afghanistan,2020-01-24,0.0,0.0,23,4,1,24,...,36.9,43.2,999.9,1018.6,34.5,0.801794,0.325375,10.4,0.17,1.0
3,4,Afghanistan,Afghanistan,2020-01-25,0.0,0.0,24,5,1,25,...,37.9,56.3,999.9,1018.0,37.8,0.728175,0.214562,6.1,0.57,1.0
4,5,Afghanistan,Afghanistan,2020-01-26,0.0,0.0,25,6,1,26,...,36.1,53.1,999.9,1014.8,33.2,0.685513,0.231656,10.8,0.00,1.0


In [41]:
test.head()

,ForecastId,Province,Country,Date,Days_After_1stJan,Dayofweek,Month,Day,Population,Density,...,dewp,fog,max,min,prcp,rh,slp,stp,temp,wdsp
0,8,Afghanistan,Afghanistan,2020-04-09,99,3,4,9,38928346,60,...,43.038347,0.313462,66.866487,47.932082,0.0,0.649365,1012.427969,947.726817,56.732486,5.603493
1,9,Afghanistan,Afghanistan,2020-04-10,100,4,4,10,38928346,60,...,42.419493,0.484523,65.181045,46.877247,0.0,0.633582,1013.464846,922.903520,55.190429,5.507524
2,10,Afghanistan,Afghanistan,2020-04-11,101,5,4,11,38928346,60,...,41.141007,0.577874,64.262317,46.074827,0.0,0.633582,1011.756430,903.631242,54.166714,5.460655
3,11,Afghanistan,Afghanistan,2020-04-12,102,6,4,12,38928346,60,...,40.944569,0.628817,63.814596,45.475912,0.0,0.633582,1010.239237,888.668657,53.513353,5.437765
4,12,Afghanistan,Afghanistan,2020-04-13,103,0,4,13,38928346,60,...,40.767912,0.656617,63.656225,45.041066,0.0,0.633582,1011.713337,877.052025,53.124613,5.426586


# Adding Index of Beds per person

In [42]:
API_beds = pd.read_csv('/kaggle/input/newest-bed-api-for-each-country/Newest_avg_bed_API.csv')

In [43]:
#merge
train = pd.merge(train,API_beds,left_on='Country',right_on='Country',how='left')
test = pd.merge(test,API_beds,left_on='Country',right_on='Country',how='left')

In [44]:
#fill NAN
train.API_beds[train.API_beds.isnull()] = np.nanmedian(train.API_beds)
test.API_beds[test.API_beds.isnull()] = np.nanmedian(test.API_beds)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
train.head()

,Id_x,Province,Country,Date,ConfirmedCases,Fatalities,Days_After_1stJan,Dayofweek,Month,Day,...,max,stp,slp,dewp,rh,ah,wdsp,prcp,fog,API_beds
0,1,Afghanistan,Afghanistan,2020-01-22,0.0,0.0,21,2,1,22,...,54.9,999.9,1024.3,27.4,0.545709,0.186448,9.4,0.00,0.0,0.5
1,2,Afghanistan,Afghanistan,2020-01-23,0.0,0.0,22,3,1,23,...,55.9,999.9,1020.8,22.8,0.461259,0.163225,14.9,99.99,1.0,0.5
2,3,Afghanistan,Afghanistan,2020-01-24,0.0,0.0,23,4,1,24,...,43.2,999.9,1018.6,34.5,0.801794,0.325375,10.4,0.17,1.0,0.5
3,4,Afghanistan,Afghanistan,2020-01-25,0.0,0.0,24,5,1,25,...,56.3,999.9,1018.0,37.8,0.728175,0.214562,6.1,0.57,1.0,0.5
4,5,Afghanistan,Afghanistan,2020-01-26,0.0,0.0,25,6,1,26,...,53.1,999.9,1014.8,33.2,0.685513,0.231656,10.8,0.00,1.0,0.5


In [46]:
test.head()

,ForecastId,Province,Country,Date,Days_After_1stJan,Dayofweek,Month,Day,Population,Density,...,fog,max,min,prcp,rh,slp,stp,temp,wdsp,API_beds
0,8,Afghanistan,Afghanistan,2020-04-09,99,3,4,9,38928346,60,...,0.313462,66.866487,47.932082,0.0,0.649365,1012.427969,947.726817,56.732486,5.603493,0.5
1,9,Afghanistan,Afghanistan,2020-04-10,100,4,4,10,38928346,60,...,0.484523,65.181045,46.877247,0.0,0.633582,1013.464846,922.903520,55.190429,5.507524,0.5
2,10,Afghanistan,Afghanistan,2020-04-11,101,5,4,11,38928346,60,...,0.577874,64.262317,46.074827,0.0,0.633582,1011.756430,903.631242,54.166714,5.460655,0.5
3,11,Afghanistan,Afghanistan,2020-04-12,102,6,4,12,38928346,60,...,0.628817,63.814596,45.475912,0.0,0.633582,1010.239237,888.668657,53.513353,5.437765,0.5
4,12,Afghanistan,Afghanistan,2020-04-13,103,0,4,13,38928346,60,...,0.656617,63.656225,45.041066,0.0,0.633582,1011.713337,877.052025,53.124613,5.426586,0.5


# Set train data for XGBoost

In [47]:
X = train.copy()
X_test = test.copy()

Province_set = set(X.Province)
Country_set = set(X.Country)

X = pd.concat([X,pd.get_dummies(X.Country)],axis=1)
X_test = pd.concat([X_test,pd.get_dummies(X_test.Country)],axis=1)
X = pd.concat([X,pd.get_dummies(X.Province)[Province_set - Country_set]],axis=1)
X_test = pd.concat([X_test,pd.get_dummies(X_test.Province)[Province_set - Country_set]],axis=1)

y_confirm = X.ConfirmedCases
y_fata = X.Fatalities


In [48]:
del X['ConfirmedCases']
del X['Fatalities']
del X['Id_x']
del X['Date']
del X['Id_y']
del X['Province']
del X['Country']

In [49]:
ForecastId = X_test.ForecastId

In [50]:
def get_percent(x):
    x = str(x)
    x = x.strip('%')
    x = float(x)/100
    return x

In [51]:
X['UrbanPopRate'] = X.UrbanPopRate.apply(lambda x:get_percent(x))
X_test['UrbanPopRate'] = X_test.UrbanPopRate.apply(lambda x:get_percent(x))

In [52]:
del X_test['ForecastId']
del X_test['Date']
del X_test['Province']
del X_test['Country']

In [53]:
X = pd.DataFrame(X,dtype=float)
X_test = pd.DataFrame(X_test,dtype=float)

In [54]:
order = X.columns
X_test = X_test[order]

In [55]:
X.head()

,Days_After_1stJan,Dayofweek,Month,Day,Population,Density,Land_Area,Migrants,MedAge,UrbanPopRate,...,Utah,Shandong,Fujian,Australian Capital Territory,Virginia,Yukon,Liaoning,Colorado,Vermont,Sichuan
0,21.0,2.0,1.0,22.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22.0,3.0,1.0,23.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23.0,4.0,1.0,24.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,24.0,5.0,1.0,25.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25.0,6.0,1.0,26.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
X_test.head()

,Days_After_1stJan,Dayofweek,Month,Day,Population,Density,Land_Area,Migrants,MedAge,UrbanPopRate,...,Utah,Shandong,Fujian,Australian Capital Territory,Virginia,Yukon,Liaoning,Colorado,Vermont,Sichuan
0,99.0,3.0,4.0,9.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100.0,4.0,4.0,10.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,101.0,5.0,4.0,11.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,102.0,6.0,4.0,12.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,103.0,0.0,4.0,13.0,38928346.0,60.0,652860.0,-62920.0,18.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# XGBoost Training

In [57]:
import xgboost as xgb

In [58]:
reg_confirm = xgb.XGBRegressor()
reg_confirm.fit(X,y_confirm)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [59]:
pred_confirm = reg_confirm.predict(X_test)

In [60]:
reg_fata = xgb.XGBRegressor()
reg_fata.fit(X,y_fata)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [61]:
pred_fata = reg_fata.predict(X_test)

In [62]:
submit = pd.DataFrame(ForecastId)
submit['ConfirmedCases']=pred_confirm
submit['Fatalities']=pred_fata

In [63]:
temp_submit_df = submit.copy()
temp_submit_df['Province'] = test.Province
temp_submit_df['Date'] = test['Date']
del temp_submit_df['ForecastId']

In [64]:
original_train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')
new_test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')

In [65]:
original_train.Date = original_train.Date.apply(lambda x:get_dt(x))
original_train.rename(columns = {'Province_State':'Province','Country_Region':'Country'},inplace = True)
for i in range(len(original_train)):
    if original_train.Province[i] is np.NaN:
        original_train.Province[i] = original_train.Country[i]
        
for i in range(len(original_train)):
    if original_train.Date[i]<datetime.strptime('2020-04-02','%Y-%m-%d') or original_train.Date[i]>datetime.strptime('2020-04-08','%Y-%m-%d'):
        original_train.drop(i,inplace=True)
        
del original_train['Id']
del original_train['Country']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [66]:
final = pd.concat([temp_submit_df,original_train],axis = 0)
final = final.sort_values(by=['Province','Date'])
final_submit = pd.DataFrame(new_test.ForecastId,columns=['ForecastId'])
final_submit['Confirmed'] = final.ConfirmedCases.values
final_submit['Fatalities'] = final.Fatalities.values

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [67]:
final_submit

,ForecastId,Confirmed,Fatalities
0,1,273.000000,6.00000
1,2,281.000000,6.00000
2,3,299.000000,7.00000
3,4,349.000000,7.00000
4,5,367.000000,11.00000
...,...,...,...
13454,13455,185.065933,30.91791
13455,13456,185.065933,30.91791
13456,13457,185.065933,30.91791
13457,13458,185.065933,30.91791


In [68]:
final_submit.to_csv('/kaggle/working/submission.csv',index=False)